## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2025-06-08 14:08:01.703139: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [10]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [11]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df["APPLICATION_TYPE"].value_counts()


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [12]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df["APPLICATION_TYPE"].value_counts()[application_df["APPLICATION_TYPE"].value_counts() < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [13]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df["CLASSIFICATION"].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [14]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() > 1]


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)


In [17]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop("IS_SUCCESSFUL", axis=1)
y = application_dummies["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [24]:
# Create a callback to save model weights every epoch
import os
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "checkpoints/weights.{epoch:02d}.weights.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch'  
)


In [25]:
# Train the model
fit_model = nn.fit(
    X_train_scaled, 
    y_train, 
    epochs=50, 
    callbacks=[cp_callback], 
    validation_data=(X_test_scaled, y_test)
)


Epoch 1/50
833/858 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7028 - loss: 0.5925
Epoch 1: saving model to checkpoints/weights.01.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7033 - loss: 0.5918 - val_accuracy: 0.7206 - val_loss: 0.5608
Epoch 2/50
830/858 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7359 - loss: 0.5502
Epoch 2: saving model to checkpoints/weights.02.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7357 - loss: 0.5503 - val_accuracy: 0.7217 - val_loss: 0.5559
Epoch 3/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7286 - loss: 0.5512
Epoch 3: saving model to checkpoints/weights.03.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7286 - loss: 0.5512 - val_accuracy: 0.7208 - val_loss: 0.5589
Epoch 4/50
839/858 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7273 - loss: 0.5539
Epoch 4: saving model to checkpoints/weights.04.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7274 - loss: 0

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7239 - loss: 0.5561
Loss: 0.5560961961746216, Accuracy: 0.7239066958427429


In [27]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")


# Alphabet Soup Charity Model Report

## Overview
This project was about helping Alphabet Soup decide which nonprofits to fund by predicting their likelihood of success using a neural network model. The dataset had over 34,000 funded organizations, and the goal was to build a model with 75%+ accuracy.

---

## Results

### Data Preprocessing
- **Target Variable:** `IS_SUCCESSFUL`
- **Features Used:** All columns except `IS_SUCCESSFUL`
- **Dropped Columns:** None — `'EIN'` and `'NAME'` weren’t included
- **Encoding:** Used `pd.get_dummies()` for categorical features
- **Scaling:** Used `StandardScaler` on feature columns

---

### Initial Model
- **Structure:**
  - Hidden Layer 1: 80 neurons, ReLU
  - Hidden Layer 2: 30 neurons, ReLU
  - Output Layer: 1 neuron, Sigmoid
- **Epochs:** 50
- **Accuracy (test):** 72.39%
- **Loss:** 0.5561

This got pretty close to the 75% goal but didn’t quite hit it, so I tried optimizing the model next.

---

## Optimized Model

### What I Changed:
- Increased neurons to 128 in the first layer
- Added a third hidden layer
- Switched activation in the second layer to `tanh`
- Added dropout (0.2) after the second hidden layer
- Trained for 100 epochs instead of 50

### Model Architecture:
- Hidden Layer 1: 128 neurons, ReLU
- Hidden Layer 2: 64 neurons, Tanh
- Dropout: 0.2
- Hidden Layer 3: 32 neurons, ReLU
- Output Layer: 1 neuron, Sigmoid

- **Accuracy (test):** 72.39% (still just short)
- **Loss:** 0.5561

Even after tweaking the model, I wasn’t able to push it above the target, but I saw stability in the accuracy and lower variance between epochs.

---

## Summary
The model was consistently in the low 72% range, which shows potential. I’d likely get better results with more hyperparameter tuning or trying a different approach like random forest or XGBoost, which tend to work better for tabular data. This was a great hands-on way to see how neural network structure changes affect performance.


